In [1]:
# -*- coding: utf-8 -*-
# 🧠 Multi-Model Image Analyzer using OpenRouter + Gradio
# Models: GPT-5 Pro | Gemini 2.5 Flash Image | Grok 4 Fast

!pip install openai gradio --quiet

from openai import OpenAI
import gradio as gr
import os

# 🔑 Put your OpenRouter API key here
OPENROUTER_API_KEY = "
"

# Optional for OpenRouter analytics (not required)
SITE_URL = "https://yourwebsite.com"
SITE_NAME = "AI Image Analyzer"

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

# ---------- 🔍 FUNCTION TO ANALYZE IMAGE ----------
def analyze_image(image):
    """
    Takes an image (from Gradio) and sends it to GPT-5 Pro, Gemini, and Grok.
    Returns text outputs from all models.
    """
    if image is None:
        return "Please upload an image.", "", ""

    # Convert Gradio image object → accessible URL or upload
    image_path = image
    image_url = image_path if image_path.startswith("http") else None

    models = {
        "GPT-5 Pro": "openai/gpt-5-pro",
        "Gemini 2.5 Flash": "google/gemini-2.5-flash-image",
        "Grok-4 Fast": "x-ai/grok-4-fast"
    }

    outputs = {}
    for name, model_id in models.items():
        try:
            completion = client.chat.completions.create(
                extra_headers={
                    "HTTP-Referer": SITE_URL,
                    "X-Title": SITE_NAME,
                },
                model=model_id,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "Describe this image in detail."},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": image_url or "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                                }
                            }
                        ]
                    }
                ]
            )
            outputs[name] = completion.choices[0].message.content
        except Exception as e:
            outputs[name] = f"⚠️ Error: {str(e)}"

    return outputs["GPT-5 Pro"], outputs["Gemini 2.5 Flash"], outputs["Grok-4 Fast"]

# ---------- 🎨 GRADIO UI ----------
with gr.Blocks(theme=gr.themes.Soft(), title="AI Image Analyzer") as demo:
    gr.Markdown(
        """
        ## 🧠 AI Image Analyzer Dashboard
        Upload an image and get insights from **GPT-5 Pro**, **Gemini 2.5**, and **Grok-4**.
        *(Powered by OpenRouter.ai)*
        """
    )

    with gr.Row():
        image_input = gr.Image(label="Upload Image", type="filepath")

    with gr.Row():
        gpt5_output = gr.Textbox(label="🧩 GPT-5 Pro Response", lines=6)
        gemini_output = gr.Textbox(label="🌈 Gemini 2.5 Flash Response", lines=6)
        grok_output = gr.Textbox(label="⚡ Grok-4 Fast Response", lines=6)

    analyze_btn = gr.Button("🔍 Analyze Image")
    analyze_btn.click(analyze_image, inputs=image_input, outputs=[gpt5_output, gemini_output, grok_output])

    gr.Markdown("---")
    gr.Markdown("⚙️ Built by Tanmay | Uses OpenRouter Multi-Model Pipeline")

# ---------- 🚀 LAUNCH ----------
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e2079943058952be9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
# -*- coding: utf-8 -*-
"""Multi-Model AI Assistant - Dark Theme"""

!pip install gradio Pillow -q

import gradio as gr
from openai import OpenAI
import requests
from PIL import Image
import base64
import io
import time

# Dark theme CSS
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

.gradio-container {
    font-family: 'Inter', sans-serif !important;
    background: linear-gradient(135deg, #0f0f23 0%, #1a1a2e 50%, #16213e 100%) !important;
    color: #ffffff !important;
}

.main-container {
    background: rgba(30, 30, 46, 0.95) !important;
    backdrop-filter: blur(10px) !important;
    border-radius: 20px !important;
    padding: 30px !important;
    margin: 20px !important;
    box-shadow: 0 20px 40px rgba(0,0,0,0.3) !important;
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
}

.header {
    text-align: center;
    margin-bottom: 30px;
}

.header h1 {
    background: linear-gradient(135deg, #ff6b6b, #ffd93d);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
    font-weight: 700;
    font-size: 2.5em;
    margin-bottom: 10px;
    text-shadow: 0 0 30px rgba(255, 107, 107, 0.3);
}

.header p {
    color: #b0b0b0;
    font-size: 1.1em;
}

.model-card {
    background: linear-gradient(135deg, #2d2d44 0%, #3d3d5c 100%) !important;
    border-radius: 15px !important;
    padding: 20px !important;
    margin: 15px 0 !important;
    border-left: 5px solid #ff6b6b !important;
    box-shadow: 0 8px 25px rgba(0,0,0,0.2) !important;
    transition: all 0.3s ease !important;
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
}

.model-card:hover {
    transform: translateY(-3px) !important;
    box-shadow: 0 15px 35px rgba(255, 107, 107, 0.2) !important;
    border-left-color: #ffd93d !important;
}

.model-header {
    display: flex;
    align-items: center;
    margin-bottom: 15px;
    padding-bottom: 10px;
    border-bottom: 2px solid rgba(255, 255, 255, 0.1);
}

.model-icon {
    font-size: 1.5em;
    margin-right: 10px;
    filter: drop-shadow(0 0 10px rgba(255, 255, 255, 0.3));
}

.model-name {
    font-weight: 600;
    color: #ffffff;
    font-size: 1.2em;
    text-shadow: 0 0 10px rgba(255, 255, 255, 0.2);
}

.model-response {
    color: #e0e0e0;
    line-height: 1.6;
    background: rgba(40, 40, 60, 0.8);
    padding: 15px;
    border-radius: 10px;
    border-left: 3px solid #ff6b6b;
    border: 1px solid rgba(255, 255, 255, 0.05);
}

.stats {
    background: linear-gradient(135deg, #ff6b6b, #ffd93d);
    color: #000000;
    padding: 8px 15px;
    border-radius: 25px;
    font-size: 0.85em;
    margin-top: 10px;
    display: inline-block;
    font-weight: 600;
    box-shadow: 0 4px 15px rgba(255, 107, 107, 0.3);
}

.upload-box {
    border: 2px dashed #ff6b6b !important;
    border-radius: 15px !important;
    padding: 20px !important;
    background: rgba(40, 40, 60, 0.8) !important;
    transition: all 0.3s ease !important;
}

.upload-box:hover {
    border-color: #ffd93d !important;
    box-shadow: 0 0 20px rgba(255, 107, 107, 0.3) !important;
}

.btn-primary {
    background: linear-gradient(135deg, #ff6b6b, #ffd93d) !important;
    border: none !important;
    border-radius: 25px !important;
    padding: 12px 30px !important;
    font-weight: 600 !important;
    transition: all 0.3s ease !important;
    color: #000000 !important;
    box-shadow: 0 8px 25px rgba(255, 107, 107, 0.3) !important;
}

.btn-primary:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 12px 30px rgba(255, 107, 107, 0.4) !important;
}

.model-selector {
    background: rgba(40, 40, 60, 0.8) !important;
    border-radius: 15px !important;
    padding: 20px !important;
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
}

.textbox, .textbox input, .textbox textarea {
    background: rgba(40, 40, 60, 0.8) !important;
    color: #ffffff !important;
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
    border-radius: 10px !important;
}

.textbox label {
    color: #ffd93d !important;
    font-weight: 600 !important;
}

.checkbox {
    color: #ffffff !important;
}

.accordion {
    background: rgba(40, 40, 60, 0.8) !important;
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
    border-radius: 10px !important;
}

.accordion label {
    color: #ffd93d !important;
    font-weight: 600 !important;
}

.footer {
    text-align: center;
    color: #b0b0b0 !important;
    border-top: 1px solid rgba(255, 255, 255, 0.1) !important;
    padding-top: 20px !important;
}
"""

def create_model_card(model_name, response, response_time, tokens_used, status):
    icon = "🤖"
    color = "#ff6b6b"
    if "GPT" in model_name:
        icon = "🚀"
        color = "#ff6b6b"
    elif "Gemini" in model_name:
        icon = "🔮"
        color = "#4ecdc4"
    elif "Grok" in model_name:
        icon = "⚡"
        color = "#ffd93d"

    status_color = "#4ecdc4" if status == "Success" else "#ff6b6b"
    status_icon = "✅" if status == "Success" else "❌"

    return f"""
    <div class="model-card" style="border-left-color: {color} !important;">
        <div class="model-header">
            <span class="model-icon">{icon}</span>
            <span class="model-name">{model_name}</span>
        </div>
        <div class="model-response">
            {response}
        </div>
        <div class="stats">
            {status_icon} Status: <span style="color: {status_color};">{status}</span> |
            ⏱️ {response_time} | 🪙 {tokens_used}
        </div>
    </div>
    """

def process_single_model(client, model_id, model_name, image, text_prompt):
    """Process image with a single model using your original code structure"""
    try:
        # Convert image using your original method
        if hasattr(image, 'save'):
            buffered = io.BytesIO()
            image.save(buffered, format="JPEG")
            image_data = base64.b64encode(buffered.getvalue()).decode('utf-8')
        else:
            # Handle numpy array from Gradio
            pil_image = Image.fromarray(image)
            buffered = io.BytesIO()
            pil_image.save(buffered, format="JPEG")
            image_data = base64.b64encode(buffered.getvalue()).decode('utf-8')

        image_url = f"data:image/jpeg;base64,{image_data}"

        start_time = time.time()

        # Your original API call structure
        completion = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "https://colab.research.google.com/",
                "X-Title": "AI Vision Assistant",
            },
            extra_body={},
            model=model_id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": text_prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url
                            }
                        }
                    ]
                }
            ]
        )

        response_time = f"{time.time() - start_time:.2f}s"
        response = completion.choices[0].message.content
        tokens_used = completion.usage.total_tokens if completion.usage else "N/A"

        return create_model_card(model_name, response, response_time, tokens_used, "Success")

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        return create_model_card(model_name, error_msg, "N/A", "N/A", "Failed")

def process_all_models(api_key, image, text_prompt, use_gpt5, use_gemini, use_grok):
    """Process image with selected models"""
    if not api_key:
        return "🔑 Please enter your OpenRouter API key first!"

    if image is None:
        return "📷 Please upload an image first!"

    # Initialize client with your original structure
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=api_key,
    )

    models_to_process = []
    if use_gpt5:
        models_to_process.append(("openai/gpt-5-pro", "GPT-5 Pro"))
    if use_gemini:
        models_to_process.append(("google/gemini-2.5-flash-image", "Gemini 2.5 Flash"))
    if use_grok:
        models_to_process.append(("x-ai/grok-4-fast", "Grok 4 Fast"))

    if not models_to_process:
        return "🤖 Please select at least one model to process!"

    results = []
    for model_id, model_name in models_to_process:
        result = process_single_model(client, model_id, model_name, image, text_prompt)
        results.append(result)

    return "".join(results)

# Create the Gradio interface
with gr.Blocks(css=custom_css, theme=gr.themes.Default(primary_hue="red")) as demo:
    with gr.Column(elem_classes="main-container"):
        with gr.Column(elem_classes="header"):
            gr.Markdown("# 🌟 AI Vision Assistant")
            gr.Markdown("### Compare multiple AI models on image analysis")

        with gr.Row():
            with gr.Column(scale=1):
                # API Key Input
                api_key = gr.Textbox(
                    label="🔑 OpenRouter API Key",
                    type="password",
                    placeholder="Enter your API key here...",
                    info="Get your key from https://openrouter.ai"
                )

                # Image Upload
                image_input = gr.Image(
                    label="📷 Upload Image",
                    type="numpy",
                    height=300,
                    elem_classes="upload-box"
                )

                # Prompt Input
                text_prompt = gr.Textbox(
                    label="💭 Your Prompt",
                    value="What is in this image? Describe it in detail.",
                    lines=3,
                    placeholder="Ask anything about the image..."
                )

                # Model Selection
                with gr.Group(elem_classes="model-selector"):
                    gr.Markdown("### 🤖 Select Models")
                    use_gpt5 = gr.Checkbox(label="GPT-5 Pro", value=True)
                    use_gemini = gr.Checkbox(label="Gemini 2.5 Flash", value=True)
                    use_grok = gr.Checkbox(label="Grok 4 Fast", value=True)

                # Process Button
                process_btn = gr.Button(
                    "🚀 Analyze Image",
                    variant="primary",
                    size="lg",
                    elem_classes="btn-primary"
                )

            with gr.Column(scale=2):
                # Results Output
                output_html = gr.HTML(
                    label="📊 Analysis Results",
                    value="<div style='text-align: center; color: #b0b0b0; padding: 40px;'>Your results will appear here after analysis...</div>"
                )

    # Examples
    with gr.Accordion("💡 Example Prompts", open=False):
        gr.Examples(
            examples=[
                "What's in this image? Describe it in detail.",
                "Analyze the composition and colors in this image.",
                "What objects can you identify in this image?",
                "Create a creative story based on this image.",
                "Describe the mood and atmosphere of this image."
            ],
            inputs=text_prompt,
            label="Click on any example to use it"
        )

    # Footer
    gr.Markdown("---", elem_classes="footer")
    gr.Markdown(
        "<div style='text-align: center; color: #b0b0b0;'>"
        "Powered by OpenRouter.ai | GPT-5 Pro • Gemini 2.5 Flash • Grok 4 Fast"
        "</div>"
    )

    # Event handling
    process_btn.click(
        fn=process_all_models,
        inputs=[api_key, image_input, text_prompt, use_gpt5, use_gemini, use_grok],
        outputs=output_html
    )

# Launch the interface
print("🎯 Launching Dark Theme AI Vision Assistant...")
demo.launch(share=True, debug=False)

🎯 Launching Dark Theme AI Vision Assistant...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec65a15c6d1138cb91.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
